Scraped the Twitter data for the '#TheKashmirFiles' using snscrape.
I scraped the data in json format then cleaned the data and finally I save the users description and tweet content data into csv files. 
I used the same data to do sentiment analysis on the user.

In [1]:
import re
import pandas as pd
import numpy as np
from pandas import json_normalize
from datetime import datetime
import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [3]:
tweets_df= pd.read_json('text-query-tweets.json',lines=True)

In [4]:
tweets_df.head()

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,...,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags
0,snscrape.modules.twitter.Tweet,https://twitter.com/TylerPaulUtt1/status/12889...,2020-07-30 23:57:02+00:00,@SiBuduh @langoinstitute do you know the Ko wo...,@SiBuduh @langoinstitute do you know the Ko wo...,1288986997143601152,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,0,...,None,NaN,None,1.288985e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,None
1,snscrape.modules.twitter.Tweet,https://twitter.com/EndlessSynthwav/status/128...,2020-07-30 23:44:04+00:00,@RockstarGames Any idea if the elephant rifle ...,@RockstarGames Any idea if the elephant rifle ...,1288983731122966534,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,None,NaN,None,NaN,None,"[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,None
2,snscrape.modules.twitter.Tweet,https://twitter.com/aanalyst50/status/12889677...,2020-07-30 22:40:40+00:00,@realDonaldTrump Trump just keeps ignoring the...,@realDonaldTrump Trump just keeps ignoring the...,1288967774795116550,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,1,...,None,NaN,None,1.288966e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,None
3,snscrape.modules.twitter.Tweet,https://twitter.com/RozzBozzy/status/128896692...,2020-07-30 22:37:18+00:00,@cslogan88 Famous 19th century song from Engla...,@cslogan88 Famous 19th century song from Engla...,1288966929236066309,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,None,NaN,None,1.288966e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...",None,None,None,None,None
4,snscrape.modules.twitter.Tweet,https://twitter.com/frryanwhitley/status/12889...,2020-07-30 22:15:36+00:00,Based on “The Tower of the Elephant” and show ...,Based on “The Tower of the Elephant” and show ...,1288961467555950592,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,1,...,None,NaN,None,1.288961e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...",None,None,None,None,None


In [5]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   _type             500 non-null    object             
 1   url               500 non-null    object             
 2   date              500 non-null    datetime64[ns, UTC]
 3   content           500 non-null    object             
 4   renderedContent   500 non-null    object             
 5   id                500 non-null    int64              
 6   user              500 non-null    object             
 7   replyCount        500 non-null    int64              
 8   retweetCount      500 non-null    int64              
 9   likeCount         500 non-null    int64              
 10  quoteCount        500 non-null    int64              
 11  conversationId    500 non-null    int64              
 12  lang              500 non-null    object             
 13  sourc

In [6]:
tweets_df.drop(columns=['_type','media','retweetedTweet','coordinates','place'],inplace=True) #deleted all the columns which has a lot of null values and will not be required for analysis.

In [7]:
tweets_df.head()

,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,...,sourceUrl,sourceLabel,outlinks,tcooutlinks,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,hashtags,cashtags
0,https://twitter.com/TylerPaulUtt1/status/12889...,2020-07-30 23:57:02+00:00,@SiBuduh @langoinstitute do you know the Ko wo...,@SiBuduh @langoinstitute do you know the Ko wo...,1288986997143601152,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,0,0,...,http://twitter.com/#!/download/ipad,Twitter for iPad,None,None,None,1.288985e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None
1,https://twitter.com/EndlessSynthwav/status/128...,2020-07-30 23:44:04+00:00,@RockstarGames Any idea if the elephant rifle ...,@RockstarGames Any idea if the elephant rifle ...,1288983731122966534,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,...,http://twitter.com/download/android,Twitter for Android,None,None,None,NaN,None,"[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None
2,https://twitter.com/aanalyst50/status/12889677...,2020-07-30 22:40:40+00:00,@realDonaldTrump Trump just keeps ignoring the...,@realDonaldTrump Trump just keeps ignoring the...,1288967774795116550,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,1,0,...,http://twitter.com/download/iphone,Twitter for iPhone,None,None,None,1.288966e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None
3,https://twitter.com/RozzBozzy/status/128896692...,2020-07-30 22:37:18+00:00,@cslogan88 Famous 19th century song from Engla...,@cslogan88 Famous 19th century song from Engla...,1288966929236066309,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,...,https://mobile.twitter.com,Twitter Web App,None,None,None,1.288966e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...",None,None,None
4,https://twitter.com/frryanwhitley/status/12889...,2020-07-30 22:15:36+00:00,Based on “The Tower of the Elephant” and show ...,Based on “The Tower of the Elephant” and show ...,1288961467555950592,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,1,0,...,http://twitter.com/download/iphone,Twitter for iPhone,None,None,None,1.288961e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...",None,None,None


We see that tweets_df has nested json field named user. We can normalize the field by using pandas function json_normalize() and can be used for better analyisis. 

In [8]:
tweets_df['user'][0]

{'_type': 'snscrape.modules.twitter.User',
 'username': 'TylerPaulUtt1',
 'id': 1283910899930210304,
 'displayname': 'Nonsensei',
 'description': 'Teacher of En·De·Fr·Zh·Ja @LanGoInstitute · Player of word · Porter of manteau #Enlightainment langoinstitute.com/langopod',
 'rawDescription': 'Teacher of En·De·Fr·Zh·Ja @LanGoInstitute · Player of word · Porter of manteau #Enlightainment https://t.co/vNamXiLAjf',
 'descriptionUrls': [{'text': 'langoinstitute.com/langopod',
   'url': 'http://langoinstitute.com/langopod',
   'tcourl': 'https://t.co/vNamXiLAjf',
   'indices': [95, 118]}],
 'verified': False,
 'created': '2020-07-16T23:46:41+00:00',
 'followersCount': 184,
 'friendsCount': 789,
 'statusesCount': 4403,
 'favouritesCount': 17861,
 'listedCount': 2,
 'mediaCount': 100,
 'location': 'Dallas, TX',
 'protected': False,
 'linkUrl': 'http://langoinstitute.com',
 'linkTcourl': 'https://t.co/NTBfIBIRSq',
 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1283911047766827008/jiKQS

In [9]:
user_data= json_normalize(tweets_df['user'])

In [10]:
user_data.head()

,_type,username,id,displayname,description,rawDescription,descriptionUrls,verified,created,followersCount,...,listedCount,mediaCount,location,protected,linkUrl,linkTcourl,profileImageUrl,profileBannerUrl,label,url
0,snscrape.modules.twitter.User,TylerPaulUtt1,1283910899930210304,Nonsensei,Teacher of En·De·Fr·Zh·Ja @LanGoInstitute · Pl...,Teacher of En·De·Fr·Zh·Ja @LanGoInstitute · Pl...,"[{'text': 'langoinstitute.com/langopod', 'url'...",False,2020-07-16T23:46:41+00:00,184,...,2,100,"Dallas, TX",False,http://langoinstitute.com,https://t.co/NTBfIBIRSq,https://pbs.twimg.com/profile_images/128391104...,None,None,https://twitter.com/TylerPaulUtt1
1,snscrape.modules.twitter.User,EndlessSynthwav,1177048009,Retrowave Enthusiast,,,None,False,2013-02-13T22:43:55+00:00,29,...,0,214,,False,None,None,https://pbs.twimg.com/profile_images/125506497...,https://pbs.twimg.com/profile_banners/11770480...,None,https://twitter.com/EndlessSynthwav
2,snscrape.modules.twitter.User,aanalyst50,799100900069543936,Don Pasha,I'm a financial consultant with an interest in...,I'm a financial consultant with an interest in...,None,False,2016-11-17T04:04:35+00:00,474,...,11,2430,,False,None,None,https://pbs.twimg.com/profile_images/799105649...,None,None,https://twitter.com/aanalyst50
3,snscrape.modules.twitter.User,RozzBozzy,1276508406686396422,RozzBozzy,"Part time indie game dev, retro gamer, full ti...","Part time indie game dev, retro gamer, full ti...",None,False,2020-06-26T13:31:47+00:00,2446,...,19,3971,England,False,None,None,https://pbs.twimg.com/profile_images/145419260...,https://pbs.twimg.com/profile_banners/12765084...,None,https://twitter.com/RozzBozzy
4,snscrape.modules.twitter.User,frryanwhitley,14585478,👻 Ryan “R.I.P.” Whitley 🎃,Ryan Whitley. Whiskey and the Weird Podcast. T...,Ryan Whitley. Whiskey and the Weird Podcast. T...,None,False,2008-04-29T14:44:25+00:00,664,...,4,828,,False,http://www.miskatonicreview.wordpress.com,https://t.co/hLPOhoqzoD,https://pbs.twimg.com/profile_images/156759278...,https://pbs.twimg.com/profile_banners/14585478...,None,https://twitter.com/frryanwhitley


In [11]:
user_data['descriptionUrls'].isnull().sum()

460

In [12]:
user_data.drop(columns=['_type','descriptionUrls','linkUrl','linkTcourl'], inplace=True) # deleting all the null values column or having same data for all the records 

In [13]:
user_data.head()

,username,id,displayname,description,rawDescription,verified,created,followersCount,friendsCount,statusesCount,favouritesCount,listedCount,mediaCount,location,protected,profileImageUrl,profileBannerUrl,label,url
0,TylerPaulUtt1,1283910899930210304,Nonsensei,Teacher of En·De·Fr·Zh·Ja @LanGoInstitute · Pl...,Teacher of En·De·Fr·Zh·Ja @LanGoInstitute · Pl...,False,2020-07-16T23:46:41+00:00,184,789,4403,17861,2,100,"Dallas, TX",False,https://pbs.twimg.com/profile_images/128391104...,None,None,https://twitter.com/TylerPaulUtt1
1,EndlessSynthwav,1177048009,Retrowave Enthusiast,,,False,2013-02-13T22:43:55+00:00,29,128,3192,2222,0,214,,False,https://pbs.twimg.com/profile_images/125506497...,https://pbs.twimg.com/profile_banners/11770480...,None,https://twitter.com/EndlessSynthwav
2,aanalyst50,799100900069543936,Don Pasha,I'm a financial consultant with an interest in...,I'm a financial consultant with an interest in...,False,2016-11-17T04:04:35+00:00,474,102,46045,12,11,2430,,False,https://pbs.twimg.com/profile_images/799105649...,None,None,https://twitter.com/aanalyst50
3,RozzBozzy,1276508406686396422,RozzBozzy,"Part time indie game dev, retro gamer, full ti...","Part time indie game dev, retro gamer, full ti...",False,2020-06-26T13:31:47+00:00,2446,2367,19975,30354,19,3971,England,False,https://pbs.twimg.com/profile_images/145419260...,https://pbs.twimg.com/profile_banners/12765084...,None,https://twitter.com/RozzBozzy
4,frryanwhitley,14585478,👻 Ryan “R.I.P.” Whitley 🎃,Ryan Whitley. Whiskey and the Weird Podcast. T...,Ryan Whitley. Whiskey and the Weird Podcast. T...,False,2008-04-29T14:44:25+00:00,664,629,5773,12848,4,828,,False,https://pbs.twimg.com/profile_images/156759278...,https://pbs.twimg.com/profile_banners/14585478...,None,https://twitter.com/frryanwhitley


In [14]:
type(user_data['created'][0])

str

In [15]:
#In the created column we have the records as string value and converting them into the date column
def extract_date(x):
  x= x[0:10]
  return datetime.strptime(x, '%Y-%m-%d').date()

In [16]:
user_data['created'] = user_data['created'].apply(extract_date)

In [17]:
user_data.head()

,username,id,displayname,description,rawDescription,verified,created,followersCount,friendsCount,statusesCount,favouritesCount,listedCount,mediaCount,location,protected,profileImageUrl,profileBannerUrl,label,url
0,TylerPaulUtt1,1283910899930210304,Nonsensei,Teacher of En·De·Fr·Zh·Ja @LanGoInstitute · Pl...,Teacher of En·De·Fr·Zh·Ja @LanGoInstitute · Pl...,False,2020-07-16,184,789,4403,17861,2,100,"Dallas, TX",False,https://pbs.twimg.com/profile_images/128391104...,None,None,https://twitter.com/TylerPaulUtt1
1,EndlessSynthwav,1177048009,Retrowave Enthusiast,,,False,2013-02-13,29,128,3192,2222,0,214,,False,https://pbs.twimg.com/profile_images/125506497...,https://pbs.twimg.com/profile_banners/11770480...,None,https://twitter.com/EndlessSynthwav
2,aanalyst50,799100900069543936,Don Pasha,I'm a financial consultant with an interest in...,I'm a financial consultant with an interest in...,False,2016-11-17,474,102,46045,12,11,2430,,False,https://pbs.twimg.com/profile_images/799105649...,None,None,https://twitter.com/aanalyst50
3,RozzBozzy,1276508406686396422,RozzBozzy,"Part time indie game dev, retro gamer, full ti...","Part time indie game dev, retro gamer, full ti...",False,2020-06-26,2446,2367,19975,30354,19,3971,England,False,https://pbs.twimg.com/profile_images/145419260...,https://pbs.twimg.com/profile_banners/12765084...,None,https://twitter.com/RozzBozzy
4,frryanwhitley,14585478,👻 Ryan “R.I.P.” Whitley 🎃,Ryan Whitley. Whiskey and the Weird Podcast. T...,Ryan Whitley. Whiskey and the Weird Podcast. T...,False,2008-04-29,664,629,5773,12848,4,828,,False,https://pbs.twimg.com/profile_images/156759278...,https://pbs.twimg.com/profile_banners/14585478...,None,https://twitter.com/frryanwhitley


In [18]:
user_data['location']=user_data['location'].replace('',np.nan)

In [21]:
user_data['location'].isnull().sum()

150

In [19]:
user_data['label'].isnull().sum()

500

In [20]:
user_data.drop(columns=['label'],inplace=True)

In [22]:
user_data.head()

,username,id,displayname,description,rawDescription,verified,created,followersCount,friendsCount,statusesCount,favouritesCount,listedCount,mediaCount,location,protected,profileImageUrl,profileBannerUrl,url
0,TylerPaulUtt1,1283910899930210304,Nonsensei,Teacher of En·De·Fr·Zh·Ja @LanGoInstitute · Pl...,Teacher of En·De·Fr·Zh·Ja @LanGoInstitute · Pl...,False,2020-07-16,184,789,4403,17861,2,100,"Dallas, TX",False,https://pbs.twimg.com/profile_images/128391104...,None,https://twitter.com/TylerPaulUtt1
1,EndlessSynthwav,1177048009,Retrowave Enthusiast,,,False,2013-02-13,29,128,3192,2222,0,214,NaN,False,https://pbs.twimg.com/profile_images/125506497...,https://pbs.twimg.com/profile_banners/11770480...,https://twitter.com/EndlessSynthwav
2,aanalyst50,799100900069543936,Don Pasha,I'm a financial consultant with an interest in...,I'm a financial consultant with an interest in...,False,2016-11-17,474,102,46045,12,11,2430,NaN,False,https://pbs.twimg.com/profile_images/799105649...,None,https://twitter.com/aanalyst50
3,RozzBozzy,1276508406686396422,RozzBozzy,"Part time indie game dev, retro gamer, full ti...","Part time indie game dev, retro gamer, full ti...",False,2020-06-26,2446,2367,19975,30354,19,3971,England,False,https://pbs.twimg.com/profile_images/145419260...,https://pbs.twimg.com/profile_banners/12765084...,https://twitter.com/RozzBozzy
4,frryanwhitley,14585478,👻 Ryan “R.I.P.” Whitley 🎃,Ryan Whitley. Whiskey and the Weird Podcast. T...,Ryan Whitley. Whiskey and the Weird Podcast. T...,False,2008-04-29,664,629,5773,12848,4,828,NaN,False,https://pbs.twimg.com/profile_images/156759278...,https://pbs.twimg.com/profile_banners/14585478...,https://twitter.com/frryanwhitley


In [23]:
def remove_special_characters(data):
  return re.sub('[^A-Za-z\s]+',' ',data)

def remove_tab_newline(data):
  return re.sub('[\n\t\s]+',' ',data)

def convert_to_lowercase(data):
  return data.lower()

In [24]:
user_data['description']= user_data['description'].apply(remove_special_characters)
user_data['description']= user_data['description'].apply(remove_tab_newline)
user_data['description']= user_data['description'].apply(convert_to_lowercase)

In [25]:
user_data.head()

,username,id,displayname,description,rawDescription,verified,created,followersCount,friendsCount,statusesCount,favouritesCount,listedCount,mediaCount,location,protected,profileImageUrl,profileBannerUrl,url
0,TylerPaulUtt1,1283910899930210304,Nonsensei,teacher of en de fr zh ja langoinstitute playe...,Teacher of En·De·Fr·Zh·Ja @LanGoInstitute · Pl...,False,2020-07-16,184,789,4403,17861,2,100,"Dallas, TX",False,https://pbs.twimg.com/profile_images/128391104...,None,https://twitter.com/TylerPaulUtt1
1,EndlessSynthwav,1177048009,Retrowave Enthusiast,,,False,2013-02-13,29,128,3192,2222,0,214,NaN,False,https://pbs.twimg.com/profile_images/125506497...,https://pbs.twimg.com/profile_banners/11770480...,https://twitter.com/EndlessSynthwav
2,aanalyst50,799100900069543936,Don Pasha,i m a financial consultant with an interest in...,I'm a financial consultant with an interest in...,False,2016-11-17,474,102,46045,12,11,2430,NaN,False,https://pbs.twimg.com/profile_images/799105649...,None,https://twitter.com/aanalyst50
3,RozzBozzy,1276508406686396422,RozzBozzy,part time indie game dev retro gamer full time...,"Part time indie game dev, retro gamer, full ti...",False,2020-06-26,2446,2367,19975,30354,19,3971,England,False,https://pbs.twimg.com/profile_images/145419260...,https://pbs.twimg.com/profile_banners/12765084...,https://twitter.com/RozzBozzy
4,frryanwhitley,14585478,👻 Ryan “R.I.P.” Whitley 🎃,ryan whitley whiskey and the weird podcast the...,Ryan Whitley. Whiskey and the Weird Podcast. T...,False,2008-04-29,664,629,5773,12848,4,828,NaN,False,https://pbs.twimg.com/profile_images/156759278...,https://pbs.twimg.com/profile_banners/14585478...,https://twitter.com/frryanwhitley


In [26]:
def remove_small_word(data):
  d=data.split(' ')
  for i in d:
    if len(i)<=2:
      d.remove(i)
  data=" ".join(d)
  return data

In [27]:
user_data['description']= user_data['description'].apply(remove_small_word)

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [29]:
stop_words = set(stopwords.words('english'))

In [30]:
def remove_stop_words(data):
  d=data.split(' ')
  for i in d:
    if i in stop_words:
      d.remove(i)
  data=" ".join(d)
  return data

In [31]:
user_data['description']= user_data['description'].apply(remove_stop_words)

In [32]:
user_data.to_csv('/content/users_description.csv',columns=['displayname','description'],index=False)

In [33]:
data= pd.read_csv('/content/users_description.csv')

In [34]:
data.head()

,displayname,description
0,Nonsensei,teacher en fr ja langoinstitute player word po...
1,Retrowave Enthusiast,NaN
2,Don Pasha,financial consultant interest finance governme...
3,RozzBozzy,part time indie game dev retro gamer full time...
4,👻 Ryan “R.I.P.” Whitley 🎃,ryan whitley whiskey weird podcast bibliotheca...


In [35]:
tweets_df['content']

0      @SiBuduh @langoinstitute do you know the Ko wo...
1      @RockstarGames Any idea if the elephant rifle ...
2      @realDonaldTrump Trump just keeps ignoring the...
3      @cslogan88 Famous 19th century song from Engla...
4      Based on “The Tower of the Elephant” and show ...
                             ...                        
495    @musafurber It definitely affected mine, as mu...
496    i can only imagine an elephant mounting anothe...
497    @jeepincj @US_Citizen_Joe @1SpencerGarrett Its...
498    @PearseDoherty Politically expedient rant! Are...
499    @ali_naka A lot of people don't want to open u...
Name: content, Length: 500, dtype: object

In [36]:
tweets_df['content']= tweets_df['content'].apply(remove_special_characters)
tweets_df['content']= tweets_df['content'].apply(remove_tab_newline)
tweets_df['content']= tweets_df['content'].apply(convert_to_lowercase)

In [37]:
tweets_df['content']= tweets_df['content'].apply(remove_small_word)

In [38]:
tweets_df['content']= tweets_df['content'].apply(remove_stop_words)

In [39]:
tweets_df.to_csv('/content/tweets_content.csv',columns=['id','content'],index=False)

In [40]:
sa = SentimentIntensityAnalyzer()

In [41]:
df= pd.read_csv('/content/tweets_content.csv')

In [42]:
sa.polarity_scores(df.loc[0]['content'])

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [43]:
df.loc[0]['content']

'sibuduh langoinstitute know word elephant its derivation one instances saw phenomenon ko'

In [44]:
df['scores']=df['content'].apply(lambda content: sa.polarity_scores(content))
df.head()

,id,content,scores
0,1288986997143601152,sibuduh langoinstitute know word elephant its ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,1288983731122966534,rockstargames idea elephant rifle bugged shoot...,"{'neg': 0.078, 'neu': 0.685, 'pos': 0.237, 'co..."
2,1288967774795116550,realdonaldtrump trump keeps ignoring elephant ...,"{'neg': 0.413, 'neu': 0.587, 'pos': 0.0, 'comp..."
3,1288966929236066309,cslogan famous century song england about elep...,"{'neg': 0.0, 'neu': 0.934, 'pos': 0.066, 'comp..."
4,1288961467555950592,based tower elephant show younger brasher cona...,"{'neg': 0.367, 'neu': 0.546, 'pos': 0.086, 'co..."


In [45]:
df['compound']=df['scores'].apply(lambda score_dict: score_dict['compound'])
df.head()

,id,content,scores,compound
0,1288986997143601152,sibuduh langoinstitute know word elephant its ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
1,1288983731122966534,rockstargames idea elephant rifle bugged shoot...,"{'neg': 0.078, 'neu': 0.685, 'pos': 0.237, 'co...",0.5951
2,1288967774795116550,realdonaldtrump trump keeps ignoring elephant ...,"{'neg': 0.413, 'neu': 0.587, 'pos': 0.0, 'comp...",-0.9451
3,1288966929236066309,cslogan famous century song england about elep...,"{'neg': 0.0, 'neu': 0.934, 'pos': 0.066, 'comp...",0.2263
4,1288961467555950592,based tower elephant show younger brasher cona...,"{'neg': 0.367, 'neu': 0.546, 'pos': 0.086, 'co...",-0.8820


In [46]:
df['overall_score']= df['compound'].apply(lambda c: 'pos' if c>=0 else 'neg' )
df.head()

,id,content,scores,compound,overall_score
0,1288986997143601152,sibuduh langoinstitute know word elephant its ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
1,1288983731122966534,rockstargames idea elephant rifle bugged shoot...,"{'neg': 0.078, 'neu': 0.685, 'pos': 0.237, 'co...",0.5951,pos
2,1288967774795116550,realdonaldtrump trump keeps ignoring elephant ...,"{'neg': 0.413, 'neu': 0.587, 'pos': 0.0, 'comp...",-0.9451,neg
3,1288966929236066309,cslogan famous century song england about elep...,"{'neg': 0.0, 'neu': 0.934, 'pos': 0.066, 'comp...",0.2263,pos
4,1288961467555950592,based tower elephant show younger brasher cona...,"{'neg': 0.367, 'neu': 0.546, 'pos': 0.086, 'co...",-0.8820,neg


In [47]:
df.groupby('overall_score').count()

,id,content,scores,compound
overall_score,,,,
neg,195,195,195,195
pos,305,305,305,305


We have more people with positive sentiments than Negative sentiments.